Prototype Version 1

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [11]:
URL = "https://www.tripadvisor.com/Attractions-g293961-Activities-c47-Sri_Lanka.html"

In [12]:
HEADERS = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36','Accept-Language': 'en-US,en;q=0.5'})

In [13]:
webpage = requests.get(URL, headers=HEADERS)

In [14]:
webpage 

<Response [403]>

Prototype Version 2

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=options)
driver.get("https://www.tripadvisor.com/Attraction_Review-g304142-d4087707-Reviews-Medilla_Beach-Tangalle_Southern_Province.html")

time.sleep(5)  # allow page to fully load

soup = BeautifulSoup(driver.page_source, "html.parser")

titles = soup.select("div[data-automation='attraction-card-title']")

for t in titles:
    print(t.get_text(strip=True))

driver.quit()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=143.0.7499.40)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff768fe88e5
	0x7ff768fe8940
	0x7ff768dc165d
	0x7ff768d991b1
	0x7ff768e497e6
	0x7ff768e6a5c2
	0x7ff768e0ac29
	0x7ff768e0ba93
	0x7ff769300640
	0x7ff7692faf80
	0x7ff7693196e6
	0x7ff769005de4
	0x7ff76900ed8c
	0x7ff768ff2004
	0x7ff768ff21b5
	0x7ff768fd7ee2
	0x7ffc51c6e8d7
	0x7ffc534ec53c


Prototype Version 3

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# ---------------- SETUP ----------------
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=options)

BASE_URL = "https://www.tripadvisor.com/Attraction_Review-g304142-d4087707-Reviews"
PLACE_SLUG = "-Medilla_Beach-Tangalle_Southern_Province.html"

all_reviews = []

# Total reviews = 86 → pages of 10 → loop 0,10,20,...,80
for offset in range(0, 90, 10):
    if offset == 0:
        url = BASE_URL + PLACE_SLUG
    else:
        url = f"{BASE_URL}-or{offset}{PLACE_SLUG}"

    print(f"Scraping page: {url}")
    driver.get(url)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    review_cards = soup.select("div[data-automation='reviewCard']")

    for card in review_cards:

        # Reviewer name
        user = card.select_one("a[data-automation='reviewerName']")
        user = user.get_text(strip=True) if user else None

        # Rating
        rating = card.select_one("svg[data-automation='bubbleRatingImage'] title")
        rating = rating.get_text(strip=True) if rating else None

        # Review title
        title = card.select_one("span.yCeTE")
        title = title.get_text(strip=True) if title else None

        # Month + Year + Category (e.g. "Mar 2016 • Couples")
        info = card.select_one("div.RpeCd")
        month_year = None
        category = None
        if info:
            parts = info.get_text(strip=True).split("•")
            if len(parts) == 2:
                month_year = parts[0].strip()
                category = parts[1].strip()

        # Review text
        review_text = card.select_one("div.biGQs._P.VImYz span.yCeTE")
        review_text = review_text.get_text(" ", strip=True) if review_text else None

        # Written date
        written_date = card.select_one("div.TreSq div.navcl")
        written_date = written_date.get_text(strip=True) if written_date else None

        all_reviews.append({
            "Reviewer Name": user,
            "Rating": rating,
            "Review Title": title,
            "Month & Year": month_year,
            "Reviewer Category": category,
            "Review Text": review_text,
            "Written Date": written_date
        })

driver.quit()

# ---------------- SAVE TO CSV ----------------
df = pd.DataFrame(all_reviews)
df.to_csv("medilla_beach_reviews.csv", index=False)

print(f"\nScraped {len(df)} reviews")
df.head()


Scraping page: https://www.tripadvisor.com/Attraction_Review-g304142-d4087707-Reviews-Medilla_Beach-Tangalle_Southern_Province.html
Scraping page: https://www.tripadvisor.com/Attraction_Review-g304142-d4087707-Reviews-or10-Medilla_Beach-Tangalle_Southern_Province.html
Scraping page: https://www.tripadvisor.com/Attraction_Review-g304142-d4087707-Reviews-or20-Medilla_Beach-Tangalle_Southern_Province.html
Scraping page: https://www.tripadvisor.com/Attraction_Review-g304142-d4087707-Reviews-or30-Medilla_Beach-Tangalle_Southern_Province.html
Scraping page: https://www.tripadvisor.com/Attraction_Review-g304142-d4087707-Reviews-or40-Medilla_Beach-Tangalle_Southern_Province.html
Scraping page: https://www.tripadvisor.com/Attraction_Review-g304142-d4087707-Reviews-or50-Medilla_Beach-Tangalle_Southern_Province.html
Scraping page: https://www.tripadvisor.com/Attraction_Review-g304142-d4087707-Reviews-or60-Medilla_Beach-Tangalle_Southern_Province.html
Scraping page: https://www.tripadvisor.com/Att

,Reviewer Name,Rating,Review Title,Month & Year,Reviewer Category,Review Text,Written Date
0,None,4 of 5 bubbles,Sunbathing,None,None,This is a beautiful beach. A good place to rel...,None
1,None,4 of 5 bubbles,Swim and relax by the beach,None,None,This stretch of beach seems wide enough and sa...,None
2,None,5 of 5 bubbles,Clean and Quiet Beach for a GREAT walk.,None,None,If you are looking for a quiet beach walk with...,None
3,None,4 of 5 bubbles,good beach to relax,None,None,"This is good, clean beach . Not crowed most of...",None
4,None,4 of 5 bubbles,Nice beaxh,None,None,Very nice beach which is very quiet. There is ...,None


Prototype Version 4

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import math

# ---------------- SETUP ----------------
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=options)

# ---------------- LIST OF URLS ----------------
URLS = [
    "https://www.tripadvisor.com/Attraction_Review-g304138-d317476-Reviews-Temple_of_the_Sacred_Tooth_Relic-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d2256431-Reviews-Bahiravokanda_Vihara_Buddha_Statue-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d6199472-Reviews-Kandy_View_Point-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d3676172-Reviews-Kandy_War_Cemetery-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d3136530-Reviews-Ceylon_Tea_Museum-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d4896053-Reviews-Kandy_City_Center-Kandy_Kandy_District_Central_Province.html",
]

MAX_REVIEWS_PER_DESTINATION = 200
all_reviews = []

for full_url in URLS:
    print(f"\nProcessing destination: {full_url}")
    driver.get(full_url)
    time.sleep(5)

for offset in range(0, 300, 10):
    if offset == 0:
        url = BASE_URL + PLACE_SLUG
    else:
        url = f"{BASE_URL}-or{offset}{PLACE_SLUG}"

    print(f"Scraping page: {url}")
    driver.get(url)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # ---------------- DESTINATION LEVEL INFO ----------------
    # Destination Title
    title_tag = soup.select_one("h1[data-test-target='mainH1']")
    destination_title = title_tag.get_text(strip=True) if title_tag else None

    # Destination Score (average rating)
    score_tag = soup.select_one("div[data-automation='bubbleRatingValue'] span")
    destination_score = score_tag.get_text(strip=True) if score_tag else None

    # Total Review counts
    total_tag = soup.select_one("div[data-automation='bubbleReviewCount'] span")
    total_reviews_text = total_tag.get_text(strip=True).replace('(', '').replace(')', '').split()[0] if total_tag else '0'
    try:
        total_reviews = int(total_reviews_text)
    except:
        total_reviews = 0

    total_reviews = min(total_reviews, MAX_REVIEWS_PER_DESTINATION)
    print(f"Total reviews to scrape for this destination: {total_reviews}")

    # Review Score (like bubbles)
    review_score_tag = soup.select_one("svg[data-automation='bubbleRatingImage'] title")
    review_score = review_score_tag.get_text(strip=True) if review_score_tag else None

    # ---------------- REVIEW LEVEL INFO ----------------
    reviews_per_page = 10
    total_pages = math.ceil(total_reviews / reviews_per_page)

    for page in range(total_pages):
        if page == 0:
            url = full_url
        else:
            url = full_url.replace("Reviews-", f"Reviews-or{page*reviews_per_page}-")

        print(f"  Scraping page: {url}")
        driver.get(url)
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        review_cards = soup.select("div[data-automation='reviewCard']")

        for card in review_cards:

            # Reviewer name
            user_tag = card.select_one("span.biGQs._P.ezezH a")
            user = user_tag.get_text(strip=True) if user_tag else None

            # Reviewer location
            location_tag = card.select_one("div.biGQs._P.navcl span")
            location = location_tag.get_text(strip=True) if location_tag else None

            # Review title
            title_tag = card.select_one("div.biGQs._P.SewaP a span")
            review_title = title_tag.get_text(strip=True) if title_tag else None

            # Month & Year + Reviewer Category
            info_tag = card.select_one("div.RpeCd")
            month_year = None
            category = None
            if info_tag:
                parts = info_tag.get_text(strip=True).split("•")
                if len(parts) == 2:
                    month_year = parts[0].strip()
                    category = parts[1].strip()

            # Review text
            review_tag = card.select_one("div._T.FKffI span.yCeTE")
            review_text = review_tag.get_text(" ", strip=True) if review_tag else None

            # Written date
            written_tag = card.select_one("div.biGQs._P.VImYz.ncFvv.navcl")
            written_date = written_tag.get_text(strip=True) if written_tag else None

            # Destination image
            img_tag = card.select_one("picture.NhWcC img")
            destination_image = img_tag['src'] if img_tag else None

            all_reviews.append({
                "Destination URL": full_url,
                "Destination Title": destination_title,
                "Destination Score": destination_score,
                "Total Reviews (Destination)": total_reviews,
                "Review Score (Bubbles)": review_score,
                "Reviewer Name": user,
                "Reviewer Location": location,
                "Review Title": review_title,
                "Month & Year": month_year,
                "Reviewer Category": category,
                "Review Text": review_text,
                "Written Date": written_date,
                "Destination Image": destination_image
            })

driver.quit()

# ---------------- SAVE TO CSV ----------------
df = pd.DataFrame(all_reviews)
df.to_csv("tripadvisor_reviews_multiple_destinations.csv", index=False)
print(f"\nTotal reviews scraped from all destinations: {len(df)}")
df.head()



Processing destination: https://www.tripadvisor.com/Attraction_Review-g304138-d317476-Reviews-Temple_of_the_Sacred_Tooth_Relic-Kandy_Kandy_District_Central_Province.html
Total reviews to scrape for this destination: 0

Processing destination: https://www.tripadvisor.com/Attraction_Review-g304138-d2256431-Reviews-Bahiravokanda_Vihara_Buddha_Statue-Kandy_Kandy_District_Central_Province.html
Total reviews to scrape for this destination: 0

Processing destination: https://www.tripadvisor.com/Attraction_Review-g304138-d6199472-Reviews-Kandy_View_Point-Kandy_Kandy_District_Central_Province.html
Total reviews to scrape for this destination: 0

Processing destination: https://www.tripadvisor.com/Attraction_Review-g304138-d3676172-Reviews-Kandy_War_Cemetery-Kandy_Kandy_District_Central_Province.html
Total reviews to scrape for this destination: 0

Processing destination: https://www.tripadvisor.com/Attraction_Review-g304138-d3136530-Reviews-Ceylon_Tea_Museum-Kandy_Kandy_District_Central_Provin

""


Prototype Version 5

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import math

# ---------------- SETUP ----------------
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=options)

# ---------------- LIST OF URLS ----------------
URLS = [
    "https://www.tripadvisor.com/Attraction_Review-g304138-d317476-Reviews-Temple_of_the_Sacred_Tooth_Relic-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d2256431-Reviews-Bahiravokanda_Vihara_Buddha_Statue-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d6199472-Reviews-Kandy_View_Point-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d3676172-Reviews-Kandy_War_Cemetery-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d3136530-Reviews-Ceylon_Tea_Museum-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d4896053-Reviews-Kandy_City_Center-Kandy_Kandy_District_Central_Province.html",
]

MAX_REVIEWS_PER_DESTINATION = 200

for full_url in URLS:
    print(f"\nProcessing destination: {full_url}")
    driver.get(full_url)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # ---------------- DESTINATION LEVEL INFO ----------------
    title_tag = soup.select_one("h1[data-test-target='mainH1']")
    destination_title = title_tag.get_text(strip=True) if title_tag else "N/A"

    score_tag = soup.select_one("div[data-automation='bubbleRatingValue'] span")
    destination_score = score_tag.get_text(strip=True) if score_tag else "N/A"

    total_tag = soup.select_one("div[data-automation='bubbleReviewCount'] span")
    total_reviews_text = total_tag.get_text(strip=True).replace('(', '').replace(')', '').split()[0] if total_tag else '0'
    try:
        total_reviews = int(total_reviews_text)
    except:
        total_reviews = 0

    total_reviews = min(total_reviews, MAX_REVIEWS_PER_DESTINATION)
    print(f"Total reviews to scrape for this destination: {total_reviews}")

    review_score_tag = soup.select_one("svg[data-automation='bubbleRatingImage'] title")
    review_score = review_score_tag.get_text(strip=True) if review_score_tag else "N/A"

    # ---------------- REVIEW LEVEL INFO ----------------
    all_reviews = []
    reviews_per_page = 10
    total_pages = math.ceil(total_reviews / reviews_per_page)

    for page in range(total_pages):
        if page == 0:
            url = full_url
        else:
            url = full_url.replace("Reviews-", f"Reviews-or{page*reviews_per_page}-")

        print(f"  Scraping page: {url}")
        driver.get(url)
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        review_cards = soup.select("div[data-automation='reviewCard']")

        for card in review_cards:
            # Reviewer name
            user_tag = card.select_one("span.biGQs._P.ezezH a")
            user = user_tag.get_text(strip=True) if user_tag else None

            # Reviewer location
            location_tag = card.select_one("div.biGQs._P.navcl span")
            location = location_tag.get_text(strip=True) if location_tag else None

            # Review title
            title_tag = card.select_one("div.biGQs._P.SewaP a span")
            review_title = title_tag.get_text(strip=True) if title_tag else None

            # Month & Year + Reviewer Category
            info_tag = card.select_one("div.RpeCd")
            month_year = None
            category = None
            if info_tag:
                parts = info_tag.get_text(strip=True).split("•")
                if len(parts) == 2:
                    month_year = parts[0].strip()
                    category = parts[1].strip()

            # Review text
            review_tag = card.select_one("div._T.FKffI span.yCeTE")
            review_text = review_tag.get_text(" ", strip=True) if review_tag else None

            # Written date
            written_tag = card.select_one("div.biGQs._P.VImYz.ncFvv.navcl")
            written_date = written_tag.get_text(strip=True) if written_tag else None

            # Destination image
            img_tag = soup.select_one("picture.NhWcC img")  # Destination image is on main page
            destination_image = img_tag['src'] if img_tag else None

            all_reviews.append({
                "Destination URL": full_url,
                "Destination Title": destination_title,
                "Destination Score": destination_score,
                "Total Reviews (Destination)": total_reviews,
                "Review Score (Bubbles)": review_score,
                "Reviewer Name": user,
                "Reviewer Location": location,
                "Review Title": review_title,
                "Month & Year": month_year,
                "Reviewer Category": category,
                "Review Text": review_text,
                "Written Date": written_date,
                "Destination Image": destination_image
            })

    # ---------------- SAVE CSV PER DESTINATION ----------------
    destination_file_name = destination_title.replace(" ", "_") + ".csv"
    df = pd.DataFrame(all_reviews)
    df.to_csv(destination_file_name, index=False)
    print(f"Saved {len(df)} reviews to {destination_file_name}")

driver.quit()



Processing destination: https://www.tripadvisor.com/Attraction_Review-g304138-d317476-Reviews-Temple_of_the_Sacred_Tooth_Relic-Kandy_Kandy_District_Central_Province.html
Total reviews to scrape for this destination: 0
Saved 0 reviews to Temple_of_the_Sacred_Tooth_RelicSomeone_from_this_business_manages_the_listing..csv

Processing destination: https://www.tripadvisor.com/Attraction_Review-g304138-d2256431-Reviews-Bahiravokanda_Vihara_Buddha_Statue-Kandy_Kandy_District_Central_Province.html
Total reviews to scrape for this destination: 0
Saved 0 reviews to Bahiravokanda_Vihara_Buddha_StatueUnclaimedIf_you_own_this_business,_claim_it_for_free_now_to_update_business_info,_respond_to_reviews,_and_more.Claim_this_listing.csv

Processing destination: https://www.tripadvisor.com/Attraction_Review-g304138-d6199472-Reviews-Kandy_View_Point-Kandy_Kandy_District_Central_Province.html
Total reviews to scrape for this destination: 0
Saved 0 reviews to Kandy_View_PointSomeone_from_this_business_man

Prototype version 6

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import math
import re
import os

# ---------------- SETUP ----------------
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(options=options)

# ---------------- LIST OF URLS ----------------
URLS = [
    "https://www.tripadvisor.com/Attraction_Review-g304138-d317476-Reviews-Temple_of_the_Sacred_Tooth_Relic-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d2256431-Reviews-Bahiravokanda_Vihara_Buddha_Statue-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d6199472-Reviews-Kandy_View_Point-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d3676172-Reviews-Kandy_War_Cemetery-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d3136530-Reviews-Ceylon_Tea_Museum-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d4896053-Reviews-Kandy_City_Center-Kandy_Kandy_District_Central_Province.html",
]

MAX_REVIEWS_PER_DESTINATION = 200

# ---------------- SCRAPING ----------------
for full_url in URLS:
    print(f"\nProcessing destination: {full_url}")
    driver.get(full_url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # ---------------- DESTINATION LEVEL INFO ----------------
    title_tag = soup.select_one("h1[data-test-target='mainH1']")
    destination_title = title_tag.get_text(strip=True) if title_tag else "Unknown"

    score_tag = soup.select_one("div[data-automation='bubbleRatingValue'] span")
    destination_score = score_tag.get_text(strip=True) if score_tag else None

    total_tag = soup.select_one("div[data-automation='bubbleReviewCount'] span")
    total_reviews_text = total_tag.get_text(strip=True).replace('(', '').replace(')', '').split()[0] if total_tag else '0'
    total_reviews = int(re.sub("[^0-9]", "", total_reviews_text))
    total_reviews = min(total_reviews, MAX_REVIEWS_PER_DESTINATION)

    print(f"Destination: {destination_title}, Total reviews to scrape: {total_reviews}")

    # ---------------- REVIEW LEVEL INFO ----------------
    reviews_per_page = 10
    total_pages = math.ceil(total_reviews / reviews_per_page)
    all_reviews = []

    for page in range(total_pages):
        if page == 0:
            url = full_url
        else:
            url = full_url.replace("Reviews-", f"Reviews-or{page*reviews_per_page}-")

        print(f"  Scraping page: {url}")
        driver.get(url)
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        review_cards = soup.select("div[data-automation='reviewCard']")

        for card in review_cards:
            # Reviewer name
            user_tag = card.select_one("span.biGQs._P.ezezH a")
            user = user_tag.get_text(strip=True) if user_tag else None

            # Reviewer location
            location_tag = card.select_one("div.biGQs._P.navcl span")
            location = location_tag.get_text(strip=True) if location_tag else None

            # Review title
            title_tag = card.select_one("h3.biGQs._P.SewaP a span.yCeTE")
            review_title = title_tag.get_text(strip=True) if title_tag else None

            # Month & Year + Reviewer Category
            info_tag = card.select_one("div.RpeCd")
            month_year = None
            category = None
            if info_tag:
                parts = info_tag.get_text(strip=True).split("•")
                if len(parts) == 2:
                    month_year = parts[0].strip()
                    category = parts[1].strip()

            # Review text
            review_tag = card.select_one("div.biGQs._P.VImYz.AWdfh span.yCeTE")
            review_text = review_tag.get_text(" ", strip=True) if review_tag else None

            # Written date
            written_tag = card.select_one("div.TreSq div.biGQs._P.VImYz.ncFvv.navcl")
            written_date = written_tag.get_text(strip=True) if written_tag else None

            all_reviews.append({
                "Destination URL": full_url,
                "Destination Title": destination_title,
                "Destination Score": destination_score,
                "Total Reviews (Destination)": total_reviews,
                "Reviewer Name": user,
                "Reviewer Location": location,
                "Review Title": review_title,
                "Month & Year": month_year,
                "Reviewer Category": category,
                "Review Text": review_text,
                "Written Date": written_date
            })

    # ---------------- SAVE TO CSV PER DESTINATION ----------------
    safe_title = re.sub(r'[\\/*?:"<>|]',"", destination_title)  # remove illegal chars
    filename = f"{safe_title}.csv"
    df = pd.DataFrame(all_reviews)
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} reviews to {filename}")

driver.quit()



Processing destination: https://www.tripadvisor.com/Attraction_Review-g304138-d317476-Reviews-Temple_of_the_Sacred_Tooth_Relic-Kandy_Kandy_District_Central_Province.html
Destination: Temple of the Sacred Tooth RelicSomeone from this business manages the listing., Total reviews to scrape: 200
  Scraping page: https://www.tripadvisor.com/Attraction_Review-g304138-d317476-Reviews-Temple_of_the_Sacred_Tooth_Relic-Kandy_Kandy_District_Central_Province.html
  Scraping page: https://www.tripadvisor.com/Attraction_Review-g304138-d317476-Reviews-or10-Temple_of_the_Sacred_Tooth_Relic-Kandy_Kandy_District_Central_Province.html
  Scraping page: https://www.tripadvisor.com/Attraction_Review-g304138-d317476-Reviews-or20-Temple_of_the_Sacred_Tooth_Relic-Kandy_Kandy_District_Central_Province.html
  Scraping page: https://www.tripadvisor.com/Attraction_Review-g304138-d317476-Reviews-or30-Temple_of_the_Sacred_Tooth_Relic-Kandy_Kandy_District_Central_Province.html
  Scraping page: https://www.tripadviso

Final Scrapping Prototype

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import math
import re
import os

# ---------------- SETUP ----------------
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(options=options)

# ---------------- LIST OF URLS ----------------
URLS = [
    "https://www.tripadvisor.com/Attraction_Review-g304138-d317476-Reviews-Temple_of_the_Sacred_Tooth_Relic-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d2256431-Reviews-Bahiravokanda_Vihara_Buddha_Statue-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d6199472-Reviews-Kandy_View_Point-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d3676172-Reviews-Kandy_War_Cemetery-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d3136530-Reviews-Ceylon_Tea_Museum-Kandy_Kandy_District_Central_Province.html",
    "https://www.tripadvisor.com/Attraction_Review-g304138-d4896053-Reviews-Kandy_City_Center-Kandy_Kandy_District_Central_Province.html",
]

MAX_REVIEWS_PER_DESTINATION = 200

# ---------------- OUTPUT FOLDER ----------------
output_folder = r"C:\NIBM\Travel recommendation system\Travel-Recommendation-system-in-Sri-Lanka-\scraped_data"
os.makedirs(output_folder, exist_ok=True)

# ---------------- SCRAPING ----------------
for full_url in URLS:
    print(f"\nProcessing destination: {full_url}")
    driver.get(full_url)
    time.sleep(5)  # allow page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # ---------------- DESTINATION LEVEL INFO ----------------
    title_tag = soup.select_one("h1[data-test-target='mainH1']")
    destination_title = title_tag.get_text(strip=True) if title_tag else "Unknown"

    # Clean the destination title to make a safe filename
    safe_title = re.sub(r'[\\/*?:"<>|]', "", destination_title)
    filename = os.path.join(output_folder, f"{safe_title}.csv")

    # ---------------- SKIP IF FILE EXISTS ----------------
    if os.path.exists(filename):
        print(f"CSV already exists for {destination_title}, skipping...")
        continue

    # ---------------- DESTINATION SCORE & TOTAL REVIEWS ----------------
    score_tag = soup.select_one("div[data-automation='bubbleRatingValue'] span")
    destination_score = score_tag.get_text(strip=True) if score_tag else None

    total_tag = soup.select_one("div[data-automation='bubbleReviewCount'] span")
    total_reviews_text = total_tag.get_text(strip=True).replace('(', '').replace(')', '').split()[0] if total_tag else '0'
    total_reviews = int(re.sub("[^0-9]", "", total_reviews_text))
    total_reviews = min(total_reviews, MAX_REVIEWS_PER_DESTINATION)

    print(f"Destination: {destination_title}, Total reviews to scrape: {total_reviews}")

    # ---------------- REVIEW LEVEL INFO ----------------
    reviews_per_page = 10
    total_pages = math.ceil(total_reviews / reviews_per_page)
    all_reviews = []

    for page in range(total_pages):
        if page == 0:
            url = full_url
        else:
            url = full_url.replace("Reviews-", f"Reviews-or{page*reviews_per_page}-")

        print(f"  Scraping page: {url}")
        driver.get(url)
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        review_cards = soup.select("div[data-automation='reviewCard']")

        for card in review_cards:
            user_tag = card.select_one("span.biGQs._P.ezezH a")
            user = user_tag.get_text(strip=True) if user_tag else None

            location_tag = card.select_one("div.biGQs._P.navcl span")
            location = location_tag.get_text(strip=True) if location_tag else None

            title_tag = card.select_one("h3.biGQs._P.SewaP a span.yCeTE")
            review_title = title_tag.get_text(strip=True) if title_tag else None

            info_tag = card.select_one("div.RpeCd")
            month_year, category = (None, None)
            if info_tag:
                parts = info_tag.get_text(strip=True).split("•")
                if len(parts) == 2:
                    month_year = parts[0].strip()
                    category = parts[1].strip()

            review_tag = card.select_one("div.biGQs._P.VImYz.AWdfh span.yCeTE")
            review_text = review_tag.get_text(" ", strip=True) if review_tag else None

            written_tag = card.select_one("div.TreSq div.biGQs._P.VImYz.ncFvv.navcl")
            written_date = written_tag.get_text(strip=True) if written_tag else None

            all_reviews.append({
                "Destination URL": full_url,
                "Destination Title": destination_title,
                "Destination Score": destination_score,
                "Total Reviews (Destination)": total_reviews,
                "Reviewer Name": user,
                "Reviewer Location": location,
                "Review Title": review_title,
                "Month & Year": month_year,
                "Reviewer Category": category,
                "Review Text": review_text,
                "Written Date": written_date
            })

    # ---------------- SAVE TO CSV PER DESTINATION ----------------
    df = pd.DataFrame(all_reviews)
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} reviews to {filename}")

driver.quit()
print("Scraping completed.")


In [2]:
import pandas as pd

In [4]:
df1= pd.read_csv(r"C:\NIBM\Travel recommendation system\Travel-Recommendation-system-in-Sri-Lanka-\scraped_data\Temple of the Sacred Tooth RelicSomeone from this business manages the listing..csv")

In [5]:
df1.head()

,Destination URL,Destination Title,Destination Score,Total Reviews (Destination),Reviewer Name,Reviewer Location,Review Title,Month & Year,Reviewer Category,Review Text,Written Date
0,https://www.tripadvisor.com/Attraction_Review-...,Temple of the Sacred Tooth RelicSomeone from t...,4.2,200,Maz,"Doha, Qatar",Disrespectful place - not for tourists,NaN,NaN,All Muslims please stay away from this evil pl...,NaN
1,https://www.tripadvisor.com/Attraction_Review-...,Temple of the Sacred Tooth RelicSomeone from t...,4.2,200,Vadim,"Murmansk, Russia",An essential part of the UNESCO website.,NaN,NaN,Despite the fundamental differences in apologe...,NaN
2,https://www.tripadvisor.com/Attraction_Review-...,Temple of the Sacred Tooth RelicSomeone from t...,4.2,200,MMJ,"Tangalle, Sri Lanka","""Temple of the Sacred Tooth Relic. A Peaceful ...",NaN,NaN,I recently visited the Temple of the Sacred To...,NaN
3,https://www.tripadvisor.com/Attraction_Review-...,Temple of the Sacred Tooth RelicSomeone from t...,4.2,200,Lou. A,"London, United Kingdom",One of the easier Temples to reach and very be...,NaN,NaN,This is our third temple and its very well lai...,NaN
4,https://www.tripadvisor.com/Attraction_Review-...,Temple of the Sacred Tooth RelicSomeone from t...,4.2,200,Wood77,"Los Angeles, California",Temple of the Sacred Tooth Relic - a Buddhist ...,NaN,NaN,"The Temple of the Sacred Tooth Relic in Kandy,...",NaN
